In [ ]:
import pyodbc

server = "10.0.40.20"
database = "DataWarehouseV2_UK"
username = "DEV_TANISH"
password = "Tanish@@1606$$"

conn_str = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={server};DATABASE={database};"
    f"UID={username};PWD={password};TrustServerCertificate=yes;"
)

tables_to_query = [
    "dbo.Vw_SI_SalesDetails",
    "dbo.Vw_SI_SalesSummary",
    "dbo.View_DiscountDetails"
]

# For each table, try to fetch up to 12 records for year 2025 (if year column exists)
try:
    conn = pyodbc.connect(conn_str, timeout=5)
    print("Connection successful!")
    cursor = conn.cursor()
    for table in tables_to_query:
        print(f"\nUp to 12 records from {table} for year 2024:")
        # Try to fetch with year filter; if fails, fetch just TOP 12
        try:
            # Check if table has a 'year' column
            cursor.execute(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table.split('.')[-1]}'")
            columns_in_table = [row[0].lower() for row in cursor.fetchall()]
            if 'year' in columns_in_table:
                query = f"SELECT TOP 12 * FROM {table} WHERE year = 2024"
            else:
                query = f"SELECT TOP 12 * FROM {table}"
            cursor.execute(query)
            columns = [column[0] for column in cursor.description]
            rows = cursor.fetchall()
            print(columns)
            for row in rows:
                print(row)
        except Exception as table_e:
            print(f"Failed to fetch from {table}: {table_e}")
    cursor.close()
    conn.close()
except Exception as e:
    print("Connection failed:", e)

Connection successful!

Up to 12 records from dbo.Vw_SI_SalesDetails for year 2024:
['CheckId', 'Date', 'Month', 'Year', 'EmployeeName', 'DayPart', 'RevenueCenter', 'MenuItem', 'CategoryId', 'Quantity', 'DiscountAmount', 'GrossAmount', 'NetAmount', 'TaxAmount', 'CompanyCode', 'SiteCode']
('T4--4718', datetime.date(2024, 5, 18), 'May', 2024, None, 'Lunch', 'Roof', 'CHICKEN SHAWARMA ROLL', '0FFE5ADF-ACB6-4AD8-B2C6-4FB640B32551', 1, Decimal('0.0000'), Decimal('5.9000'), Decimal('5.3640'), Decimal('0.5360'), 'C2116', 'L3049')
('83947', datetime.date(2024, 9, 20), 'September', 2024, 'Oswald Ndlovu', 'Dinner', "Hurricane's Grill Dine I", 'Pepper Steak', '51C7F3C0-E793-4A00-92AF-15E56D04CDC6', 1, Decimal('0.0000'), Decimal('210.0000'), Decimal('187.5000'), Decimal('22.5000'), 'C1642', 'L2328')
('306054', datetime.date(2024, 12, 9), 'December', 2024, 'Kerim Sassi', 'Lunch', 'Pisco Bar', 'POS 1', '58A8E651-87DA-40C5-B012-2D5928E41337', 1, Decimal('0.0000'), Decimal('0.0000'), Decimal('0.0000'),

In [ ]:
import pyodbc

server = "10.0.40.20"
database = "DataWarehouseV2_UK"
username = "DEV_TANISH"
password = "Tanish@@1606$$"

conn_str = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={server};DATABASE={database};"
    f"UID={username};PWD={password};TrustServerCertificate=yes;"
)

company_code_to_find = "C1587"
site_code_to_find = "L2312"

try:
    conn = pyodbc.connect(conn_str, timeout=5)
    print("Connection successful!")
    cursor = conn.cursor()

    # Get all tables and views in the database
    cursor.execute("""
        SELECT TABLE_SCHEMA, TABLE_NAME, TABLE_TYPE
        FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_TYPE IN ('BASE TABLE', 'VIEW')
        ORDER BY TABLE_SCHEMA, TABLE_NAME
    """)
    objects = cursor.fetchall()

    found_tables = []

    # Only print those where we have the specified CompanyCode and SiteCode
    for schema, table, obj_type in objects:
        full_name = f"{schema}.{table}"

        # Check if both 'CompanyCode' and 'SiteCode' columns exist in this table/view
        cursor.execute("""
            SELECT COLUMN_NAME
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_SCHEMA = ? AND TABLE_NAME = ? AND COLUMN_NAME IN ('CompanyCode', 'SiteCode')
        """, (schema, table))
        columns = [row[0] for row in cursor.fetchall()]
        if "CompanyCode" in columns and "SiteCode" in columns:
            # Try to find at least one row with the given company and site code
            try:
                query = f"""
                    SELECT TOP 1 CompanyCode, SiteCode
                    FROM {full_name}
                    WHERE CompanyCode = ? AND SiteCode = ?
                """
                cursor.execute(query, (company_code_to_find, site_code_to_find))
                result = cursor.fetchone()
                if result:
                    print(f"✅ {obj_type}: {full_name} contains CompanyCode='{company_code_to_find}' and SiteCode='{site_code_to_find}'")
                    found_tables.append(full_name)
                # Do not print anything if not found
            except Exception as qerr:
                # Do not print anything if error
                pass
        # Do not print anything if columns not present

    if found_tables:
        print("\nSummary: The following tables/views contain the specified CompanyCode and SiteCode:")
        for t in found_tables:
            print(" -", t)
    else:
        print("\nNo tables or views found with both columns and the specified values.")

    cursor.close()
    conn.close()
except Exception as e:
    print("Connection failed:", e)

Connection successful!
✅ BASE TABLE: dbo.CashupCategoryPaperchaseCategoryMapping contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.CoverSourceMst contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.DayPartMst contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.DiscountTypeMst contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.EmployeeMst contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.EposPaperchaseCategoryWeeklySubCategoryMapping contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.EposSubCategoryMstMapping contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.KPI_CheckWiseSummaryDetails contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.KPI_CoverSourceDetails contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.KPI_DiscountDetails contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.KPI_ItemWiseSalesDetails contains Company

In [6]:
import pyodbc

server = "10.0.40.20"
database = "DataWarehouseV2_UK"
username = "DEV_TANISH"
password = "Tanish@@1606$$"

conn_str = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={server};DATABASE={database};"
    f"UID={username};PWD={password};TrustServerCertificate=yes;"
)

# Use TOP 100 instead of LIMIT 100 for SQL Server
query = """
 SELECT MenuItem, FORMAT(SUM(NetAmount), 'N0', 'en-GB') AS TotalSales
FROM dbo.Vw_SI_SalesDetails
WHERE Year = 2025 AND CompanyCode = 'C1587' AND SiteCode = 'L2312'
GROUP BY MenuItem
ORDER BY FORMAT(SUM(NetAmount), 'N0', 'en-GB') DESC
"""

try:
    conn = pyodbc.connect(conn_str, timeout=5)
    cursor = conn.cursor()
    cursor.execute(query)
    columns = [column[0] for column in cursor.description]
    rows = cursor.fetchall()
    print(columns)
    for row in rows:
        print(row)
    cursor.close()
    conn.close()
except Exception as e:
    print("Query failed:", e)

['MenuItem', 'TotalSales']
('Fleur De Pedesclaux 17', '996')
('Bienvenue Batard Montrachet Faivelay 2021', '995')
('Dom Perignon JM Basquiat Yellow 2015', '992')
('Gl Krug Rose Nv', '990')
('Gl Tawny Port', '986')
('Dom Perignon Rose 2002', '983')
('Couronne des rois 2025', '983')
('Stolichnaya ELITE', '98')
('Talisker 10Y', '98')
('Mag Macon Ige Leflaive 2023', '979')
('Car Château La Coste Rosé 2023', '978')
('Opus One 2021', '977')
('2023 Crozes Hermitage Blanc A. Graillot', '975')
('Mag Chateau Simone Rouge 2019', '972')
('Burrata tartine', '971')
('Chateau Argadens Blanc 2022', '970')
('Gratin', '97,959')
('Tomato Gnocchi', '97,247')
('25ml Grey Goose', '97')
('Tanqueray 10', '968')
('2016 Penfolds Grange Bin 95', '967')
('Domaine De Trevallon Rouge * 2020', '961')
('Lentil salad', '96,637')
('Chips', '96,391')
('Clase Azul Anejo', '96')
('Glenfiddich 12Y RR', '96')
('Dom Perignon P2 2004', '953')
('Del Maguey Chichicapa RR', '95')
('Clase Azul Anejo RR', '95')
('Chateau Palmer 20

In [ ]:
import pyodbc

server = "10.0.40.20"
database = "DataWarehouseV2_UK"
username = "DEV_TANISH"
password = "Tanish@@1606$$"

conn_str = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={server};DATABASE={database};"
    f"UID={username};PWD={password};TrustServerCertificate=yes;"
)

# Query to get CompanyCode and SiteCode pairs that are present in all three tables
query = """
SELECT cm.CompanyCode, cm.SiteCode
FROM dbo.View_DiscountDetails cm
INNER JOIN dbo.Vw_SI_SalesSummary ss
    ON cm.CompanyCode = ss.CompanyCode AND cm.SiteCode = ss.SiteCode
INNER JOIN dbo.Vw_SI_SalesDetails sd
    ON cm.CompanyCode = sd.CompanyCode AND cm.SiteCode = sd.SiteCode
GROUP BY cm.CompanyCode, cm.SiteCode
ORDER BY cm.CompanyCode, cm.SiteCode;
"""

try:
    conn = pyodbc.connect(conn_str, timeout=5)
    cursor = conn.cursor()
    cursor.execute(query)
    columns = [column[0] for column in cursor.description]
    rows = cursor.fetchall()
    print(columns)
    for row in rows:
        print(row)
    cursor.close()
    conn.close()
except Exception as e:
    print("Query failed:", e)


Query failed: ('42S22', "[42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name 'SiteCode'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name 'SiteCode'. (207); [42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name 'SiteCode'. (207); [42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name 'SiteCode'. (207); [42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name 'SiteCode'. (207)")


In [4]:
import pyodbc

server = "10.0.40.20"
database = "DataWarehouseV2_UK"
username = "DEV_TANISH"
password = "Tanish@@1606$$"

conn_str = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={server};DATABASE={database};"
    f"UID={username};PWD={password};TrustServerCertificate=yes;"
)

query = """
SELECT TOP 2
    Month,
    Covers
FROM dbo.Vw_SI_SalesSummary
WHERE Month IN (
    'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August'
)
ORDER BY
    CASE Month
        WHEN 'January' THEN 1
        WHEN 'February' THEN 2
        WHEN 'March' THEN 3
        WHEN 'April' THEN 4
        WHEN 'May' THEN 5
        WHEN 'June' THEN 6
        WHEN 'July' THEN 7
        WHEN 'August' THEN 8
        WHEN 'September' THEN 9
        WHEN 'October' THEN 10
        WHEN 'November' THEN 11
        WHEN 'December' THEN 12
        ELSE 13
    END
"""

try:
    conn = pyodbc.connect(conn_str, timeout=5)
    cursor = conn.cursor()
    cursor.execute(query)
    columns = [column[0] for column in cursor.description]
    rows = cursor.fetchall()
    print(columns)
    for row in rows:
        print(row)
    cursor.close()
    conn.close()
except Exception as e:
    print("Query failed:", e)


['Month', 'Covers']
('January', 1)
('January', 2)


In [ ]:


query = """
SELECT CompanyCode, CompanyName
FROM dbo.View_DiscountDetails
"""

try:
    conn = pyodbc.connect(conn_str, timeout=5)
    cursor = conn.cursor()
    cursor.execute(query)
    columns = [column[0] for column in cursor.description]
    rows = cursor.fetchall()
    print(columns)
    for row in rows:
        print(row)
    cursor.close()
    conn.close()
except Exception as e:
    print("Query failed:", e)


['CompanyCode', 'CompanyName']
('C1030', 'Mission Sandwich Social')
('C1645', 'Ten Lincoln Ltd')
('C1665', 'Taka Mayfair Ltd')
('C1674', 'The Swan Inn Wychwood Ltd (NEWI)')
('C1776', 'The Medlar Restaurant LTD')
('C1801', 'Restaurant Craft Ltd')
('C1812', 'Coya Mykonos S.A')
('C1936', 'Montagu House LTD')
('C1948', 'Roka Restaurant LLC')
('C2091', 'Kolamba Carnaby Ltd')
('C2137', 'OMA RESTAURANT LTD')
('C2178', 'ROE RESTAURANT LTD')
('C2181', 'The Coal Office Restaurant Limited')
('C2182', 'NOZA EVENTS MANAGEMENT FZE LLC')
('C1361', 'Sunset Beach Hotel')
('C1432', 'Double Shot TB Upper West Side, LLC')
('C1434', 'And Pizza')
('C1644', 'DININGS LONDON LIMITED')
('C1738', 'Dinings SW3 Ltd')
('C1764', 'ADVANCED HORIZON TRADING COMPANY')
('C1782', 'CL14 Ltd')
('C1806', 'LPM Abu Dhabi')
('C1807', 'LPM Hong Kong')
('C1843', 'LPM Riyadh')
('C1874', 'D Grande Ltd')
('C1915', 'Kipferl Patisserie Ltd')
('C2029', 'Sunbeam Photographic Limited')
('C2152', 'EatPiu Limited')
('C2154', 'Mangerplus Li

In [10]:
import pyodbc

queries = [
    # ("DayPartName", "SELECT DISTINCT DayPartName FROM DayPartMst WHERE CompanyCode='C1587' AND SiteCode='L2312'"),
    # ("CategoryName", "SELECT DISTINCT PaperchaseCategoryName FROM PaperchaseCategoryMaster WHERE CompanyCode='C1587' AND SiteCode='L2312'"),
    # ("SubCategoryName", "SELECT DISTINCT SubCategoryName FROM MenuItemCategoryMst WHERE CompanyCode='C1587' AND SiteCode='L2312'"),
    # ("RevenueCenterName", "SELECT DISTINCT RevenueCenterName FROM RevenueCenterMst WHERE CompanyCode='C1587' AND SiteCode='L2312'"),
    ("MenuItem", "SELECT DISTINCT * FROM Vw_SI_CategoryDetails WHERE CompanyCode='C1587' AND SiteCode='L2312'")
]

try:
    conn = pyodbc.connect(conn_str, timeout=5)
    cursor = conn.cursor()
    for label, query in queries:
        print(f"\nResults for {label}:")
        cursor.execute(query)
        columns = [column[0] for column in cursor.description]
        rows = cursor.fetchall()
        print(columns)
        for row in rows:
            print(row)
    cursor.close()
    conn.close()
except Exception as e:
    print("Query failed:", e)


Results for MenuItem:
Query failed: ('42S22', "[42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name 'CompanyCode'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name 'SiteCode'. (207)")
